In [70]:
# Warning control
import warnings
import os

warnings.filterwarnings("ignore")

from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## Tools


In [71]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()  # allow to search internet via google
scrape_tool = ScrapeWebsiteTool()  ## scrape those websites

## agent


In [72]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements and share the google maps link for the venue, if the user says the link is not working, find a new location",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
)

In [73]:
# Agent 2: Logistics Manager
logistics_manager = Agent(
    role="Logistics Manager",
    goal=("Manage all logistics for the event " "including catering and equipmen"),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
)

In [74]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and " "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
)

## Venue details using the pydantic object


In [75]:
from pydantic import BaseModel


# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str
    google_maps_link: str


## so this is how the agent is suppose to answer or give the ouput about the venue

## Creating Tasks

- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.


In [76]:
venue_task = Task(
    description="Find a venue in {event_city} "
    "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    # Outputs the venue details as a JSON file
    agent=venue_coordinator,
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.


In [77]:
logistics_task = Task(
    description="Coordinate catering and "
    "equipment for an event "
    "with {expected_participants} participants "
    "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager,
)

In [78]:
marketing_task = Task(
    description="Promote the {event_topic} "
    "aiming to engage at least"
    "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,
)

## Creating the CREW


In [79]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True,
)

2025-04-12 20:59:50,930 - 8234502208 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


## Running the CREW


In [80]:
event_details = {
    "event_topic": "Friendly Meetup",
    "event_description": "A gathering of friends for coffee " "to have a good time.",
    "event_city": "Sandy Springs, Atlanta, Georgia",
    "tentative_date": "2025-04-15",
    "expected_participants": 20,
    "budget": 1500,
    "venue_type": "restaurant",
}

In [81]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Sandy Springs, Atlanta, Georgia that meets criteria for Friendly Meetup.


> Entering new CrewAgentExecutor chain...
Thought: To find the appropriate venue in Sandy Springs, Atlanta, Georgia for a Friendly Meetup, I need to first search the internet for venues in that area that could accommodate such an event.

Action: Search the internet
Action Input: {"search_query": "Friendly Meetup venues in Sandy Springs, Atlanta, Georgia"} 


Search results: Title: 36 Best Meetup Venues for Rent in Sandy Springs, GA | Peerspace
Link: https://www.peerspace.com/venues/sandy-springs--ga/meetup-venue
Snippet: Easily rent a meetup event venue in Sandy Springs, GA. Access a collection of unique, undiscovered locations and make your event a meaningful one.
---
Title: Rent a meetup venue in Sandy Springs, Atlanta, GA - Peerspace
Link: https://www.peerspace.com/neighborhood/sandy-springs--atlanta--ga/meetup-venue
Snip

Thought: To organize the catering and equipment setup, I need to understand the specific requirements and preferences for the event. I should start by determining the catering options available near the venue and check if any particular setup is required for the equipment at the venue.

Action: Search the internet
Action Input: {"search_query": "catering services near Sandy Springs, Atlanta, Georgia"} 


Search results: Title: Three Sisters Catering: Home
Link: https://threesisterscatering.com/
Snippet: Three Sisters Catering is where creativity extends to all aspects of social and corporate catering. From Boxed Meals, Catering-to-Go, to Weddings, Mitzvahs and ...
---
Title: THE Best 10 CATERERS in SANDY SPRINGS, GA - Yelp
Link: https://www.yelp.com/search?cflt=catering&find_loc=Sandy+Springs%2C+GA
Snippet: Top 10 Best Caterers Near Sandy Springs, Georgia - With Real Reviews · 1. Three Sisters Catering · 2. Ooey Gooey Catering · 3. Talk Of The Town · 4. The Portable ...
---
Title: Cate

In [82]:
import json
from pprint import pprint

with open("venue_details.json") as f:
    data = json.load(f)

pprint(data)

{'address': 'Atlanta\'s Premier Class "A" Medical Office Buildings, Sandy '
            'Springs, Atlanta, Georgia',
 'booking_status': 'Available',
 'capacity': 100,
 'google_maps_link': 'https://www.peerspace.com/venues/sandy-springs--ga/meetup-venue',
 'name': "Sandy Spring's Office with a View"}
